In [45]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

orders schema = order_id, order_date, order_customer_id, order_status

In [46]:
orders_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/orders/*")

order_items schema = order_item_id, order_id, order_item_product_id, order_item_quantity, order_item_subtotal, order_item_product_price

In [47]:
order_items_rdd=spark.sparkContext.textFile("/public/trendytech/retail_db/order_items/*")

customers schema = customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode

In [48]:
customers_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/customers/*")

In [49]:
customers_rdd.take(5)

['1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521',
 '2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126',
 '3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725',
 '4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069',
 '5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,"10 Crystal River Mall ",Caguas,PR,00725']

In [50]:
orders_rdd.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [51]:
order_items_rdd.take(5)

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

In [52]:
# top 10 customers who who spent the most

In [53]:
order_items_map = order_items_rdd.map(lambda x:((int(x.split(',')[1]),float(x.split(',')[4]))))
order_items_map.take(5)

[(1, 299.98), (2, 199.99), (2, 250.0), (2, 129.99), (4, 49.98)]

In [54]:
orders_map = orders_rdd.map(lambda x:(int(x.split(',')[0]),(int(x.split(',')[2]))))
orders_map.take(5)

[(1, 11599), (2, 256), (3, 12111), (4, 8827), (5, 11318)]

In [55]:
join_rdd = order_items_map.join(orders_map)
join_rdd.take(5)

[(35212, (49.98, 8774)),
 (35212, (299.97, 8774)),
 (35212, (249.9, 8774)),
 (35212, (49.98, 8774)),
 (35212, (149.94, 8774))]

In [56]:
mapped_rdd = join_rdd.map(lambda x:(x[1][1],x[1][0]))
mapped_rdd.take(5)

[(8774, 49.98), (8774, 299.97), (8774, 249.9), (8774, 49.98), (8774, 149.94)]

In [57]:
# Reduce by key customer
reduced_rdd = mapped_rdd.reduceByKey(lambda x,y : x+y).sortBy(lambda x : x[1],ascending=False)
reduced_rdd.take(10)


[(791, 10524.169999999996),
 (9371, 9299.029999999999),
 (8766, 9296.14),
 (1657, 9223.71),
 (2641, 9130.92),
 (1288, 9019.11),
 (3710, 9019.099999999999),
 (4249, 8918.85),
 (5654, 8904.95),
 (5624, 8761.98)]

In [58]:
# no of csuomters from Caguas city

In [59]:
mapped_rdd = customers_rdd.map(lambda x:x.split(",")[6])

In [60]:
mapped_rdd.take(5)

['Brownsville', 'Littleton', 'Caguas', 'San Marcos', 'Caguas']

In [61]:
final_count = mapped_rdd.filter( lambda x :  x== 'Caguas').count()

In [62]:
final_count

4584

In [63]:
# Top 3 states with maximum customers

In [65]:
mapped_rdd = customers_rdd.map(lambda x : (x.split(",")[7],1)).reduceByKey(lambda x,y : x+y).sortBy(lambda x : x[1],ascending = False).take(3)
mapped_rdd

[('PR', 4771), ('CA', 2012), ('NY', 775)]

In [66]:
# no of Customers_haivng more than $1000 in total

In [67]:
no = join_rdd.map(lambda x:(x[1][1],x[1][0])).reduceByKey(lambda x,y:x+y).filter(lambda x : x[1] > 1000).count()
no

11148

In [68]:
# state with most number of closed status

In [69]:
orders_map = orders_rdd.map(lambda x:((int(x.split(',')[2])),(x.split(',')[3]))).filter(lambda x :x[1]=='CLOSED')

In [70]:
orders_map.take(5)

[(11599, 'CLOSED'),
 (8827, 'CLOSED'),
 (1837, 'CLOSED'),
 (1205, 'CLOSED'),
 (11441, 'CLOSED')]

In [71]:
customers_map = customers_rdd.map(lambda x :(int(x.split(',')[0]),x.split(',')[7]))
customers_map.take(5)

[(1, 'TX'), (2, 'CO'), (3, 'PR'), (4, 'CA'), (5, 'PR')]

In [73]:
join_data = orders_map.join(customers_map).map(lambda x : (x[1][1],1)).reduceByKey(lambda x,y :x+y).sortBy(lambda x : x[0],ascending = False)
join_data.take(5)

[('WV', 15), ('WI', 44), ('WA', 47), ('VA', 97), ('UT', 45)]